# Create data

In [1]:
import psycopg
from faker import Faker
import random
from datetime import datetime, timedelta

fake = Faker()

# -----------------------------------------
# 1. Connection helper
# -----------------------------------------
def get_connection():
    return psycopg.connect(
        host="localhost",
        port=5431,  # from docker-compose mapping
        dbname="test_db",
        user="luke",
        password="luke_password"
    )


In [15]:
def drop_all_tables():
    with get_connection() as conn:
        cur = conn.cursor()
        
        # Drop tables in correct order due to foreign key constraints
        tables = [
            "marketing_campaigns",
            "interactions",
            "invoices",
            "sales",
            "opportunities",
            "leads",
            "products",
            "suppliers",
            "employees",
            "customers"
        ]
        
        for table in tables:
            cur.execute(f"DROP TABLE IF EXISTS {table} CASCADE")
        
        conn.commit()
        print("✅ All tables dropped successfully.")

# Call the function
drop_all_tables()

✅ All tables dropped successfully.


In [16]:
# -----------------------------------------
# 2. Create schema
# -----------------------------------------
def create_crm_tables():
    with get_connection() as conn:
        cur = conn.cursor()

        cur.execute("""
        CREATE TABLE IF NOT EXISTS customers (
            customer_id SERIAL PRIMARY KEY,
            first_name VARCHAR(100),
            last_name VARCHAR(100),
            email VARCHAR(255),
            phone VARCHAR(50),
            address VARCHAR(255),
            city VARCHAR(100),
            country VARCHAR(100),
            signup_date DATE,
            loyalty_points INT
        );
        """)

        cur.execute("""
        CREATE TABLE IF NOT EXISTS suppliers (
            supplier_id SERIAL PRIMARY KEY,
            supplier_name VARCHAR(150),
            contact_person VARCHAR(100),
            phone VARCHAR(50),
            email VARCHAR(255),
            address VARCHAR(255),
            city VARCHAR(100),
            country VARCHAR(100),
            rating DECIMAL(3,2),
            payment_terms VARCHAR(50),
            active_since DATE
        );
        """)

        cur.execute("""
        CREATE TABLE IF NOT EXISTS products (
            product_id SERIAL PRIMARY KEY,
            product_name VARCHAR(150),
            sku VARCHAR(50),
            description VARCHAR(500),
            category VARCHAR(100),
            unit_price DECIMAL(10,2),
            supplier_id INT REFERENCES suppliers(supplier_id),
            stock_quantity INT,
            reorder_level INT,
            discontinued BOOLEAN
        );
        """)

        cur.execute("""
        CREATE TABLE IF NOT EXISTS employees (
            employee_id SERIAL PRIMARY KEY,
            first_name VARCHAR(100),
            last_name VARCHAR(100),
            email VARCHAR(255),
            phone VARCHAR(50),
            department VARCHAR(100),
            position VARCHAR(100),
            hire_date DATE,
            salary DECIMAL(10,2),
            is_active BOOLEAN
        );
        """)

        cur.execute("""
        CREATE TABLE IF NOT EXISTS leads (
            lead_id SERIAL PRIMARY KEY,
            lead_name VARCHAR(150),
            company VARCHAR(150),
            email VARCHAR(255),
            phone VARCHAR(50),
            status VARCHAR(50),
            source VARCHAR(100),
            assigned_to INT REFERENCES employees(employee_id),
            created_on DATE,
            converted BOOLEAN
        );
        """)

        cur.execute("""
        CREATE TABLE IF NOT EXISTS opportunities (
            opportunity_id SERIAL PRIMARY KEY,
            lead_id INT REFERENCES leads(lead_id),
            product_id INT REFERENCES products(product_id),
            estimated_value DECIMAL(12,2),
            stage VARCHAR(100),
            expected_close DATE,
            assigned_to INT REFERENCES employees(employee_id),
            created_on DATE,
            notes VARCHAR(500),
            won BOOLEAN
        );
        """)

        cur.execute("""
        CREATE TABLE IF NOT EXISTS sales (
            sale_id SERIAL PRIMARY KEY,
            customer_id INT REFERENCES customers(customer_id),
            employee_id INT REFERENCES employees(employee_id),
            product_id INT REFERENCES products(product_id),
            quantity INT,
            total_amount DECIMAL(12,2),
            sale_date DATE,
            payment_method VARCHAR(50),
            region VARCHAR(100),
            discount DECIMAL(5,2)
        );
        """)

        cur.execute("""
        CREATE TABLE IF NOT EXISTS invoices (
            invoice_id SERIAL PRIMARY KEY,
            sale_id INT REFERENCES sales(sale_id),
            invoice_number VARCHAR(50),
            issue_date DATE,
            due_date DATE,
            total_amount DECIMAL(12,2),
            tax DECIMAL(5,2),
            status VARCHAR(50),
            billing_address VARCHAR(255),
            payment_received BOOLEAN
        );
        """)

        cur.execute("""
        CREATE TABLE IF NOT EXISTS interactions (
            interaction_id SERIAL PRIMARY KEY,
            customer_id INT REFERENCES customers(customer_id),
            employee_id INT REFERENCES employees(employee_id),
            interaction_type VARCHAR(100),
            channel VARCHAR(50),
            interaction_date DATE,
            notes VARCHAR(500),
            follow_up_required BOOLEAN,
            satisfaction_score INT,
            duration_minutes INT,
            outcome VARCHAR(100)
        );
        """)

        cur.execute("""
        CREATE TABLE IF NOT EXISTS marketing_campaigns (
            campaign_id SERIAL PRIMARY KEY,
            campaign_name VARCHAR(150),
            start_date DATE,
            end_date DATE,
            budget DECIMAL(12,2),
            channel VARCHAR(100),
            target_audience VARCHAR(255),
            conversion_rate DECIMAL(5,2),
            responsible_employee INT REFERENCES employees(employee_id),
            active BOOLEAN
        );
        """)

        conn.commit()
        print("✅ All CRM tables created successfully.")


# -----------------------------------------
# 3. Populate with fake data
# -----------------------------------------
def populate_crm_data(num_records=100):
    with get_connection() as conn:
        cur = conn.cursor()

        # Suppliers
        for _ in range(num_records):
            cur.execute("""
                INSERT INTO suppliers (supplier_name, contact_person, phone, email, address, city, country, rating, payment_terms, active_since)
                VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
            """, (
                fake.company(),
                fake.name(),
                fake.phone_number(),
                fake.company_email(),
                fake.address(),
                fake.city(),
                fake.country(),
                round(random.uniform(2.0, 5.0), 2),
                random.choice(["Net 30", "Net 60", "Advance"]),
                fake.date_this_decade()
            ))

        # Employees
        for _ in range(num_records):
            cur.execute("""
                INSERT INTO employees (first_name, last_name, email, phone, department, position, hire_date, salary, is_active)
                VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
            """, (
                fake.first_name(),
                fake.last_name(),
                fake.company_email(),
                fake.phone_number(),
                random.choice(["Sales", "Support", "HR", "IT", "Finance"]),
                random.choice(["Manager", "Executive", "Lead", "Associate"]),
                fake.date_this_decade(),
                round(random.uniform(40000, 120000), 2),
                fake.boolean()
            ))

        # Customers
        for _ in range(num_records):
            cur.execute("""
                INSERT INTO customers (first_name, last_name, email, phone, address, city, country, signup_date, loyalty_points)
                VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
            """, (
                fake.first_name(),
                fake.last_name(),
                fake.email(),
                fake.phone_number(),
                fake.address(),
                fake.city(),
                fake.country(),
                fake.date_this_decade(),
                random.randint(0, 5000)
            ))

        # Products
        for _ in range(num_records):
            cur.execute("""
                INSERT INTO products (product_name, sku, description, category, unit_price, supplier_id, stock_quantity, reorder_level, discontinued)
                VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
            """, (
                fake.word().capitalize() + " " + fake.word().capitalize(),
                fake.bothify(text='SKU-#####'),
                fake.text(max_nb_chars=200),
                random.choice(["Software", "Hardware", "Service", "Subscription"]),
                round(random.uniform(10, 2000), 2),
                random.randint(1, num_records),
                random.randint(10, 500),
                random.randint(5, 50),
                fake.boolean()
            ))

        # Leads
        for _ in range(num_records):
            cur.execute("""
                INSERT INTO leads (lead_name, company, email, phone, status, source, assigned_to, created_on, converted)
                VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
            """, (
                fake.name(),
                fake.company(),
                fake.email(),
                fake.phone_number(),
                random.choice(["New", "Contacted", "Qualified", "Lost"]),
                random.choice(["Web", "Referral", "Cold Call", "Event"]),
                random.randint(1, num_records),
                fake.date_this_decade(),
                fake.boolean()
            ))

        conn.commit()
        print("✅ Basic CRM data populated successfully.")


# -----------------------------------------
# Run both steps
# -----------------------------------------
if __name__ == "__main__":
    create_crm_tables()
    populate_crm_data(50)


✅ All CRM tables created successfully.
✅ Basic CRM data populated successfully.


In [2]:
def extract_schema():
    conn = get_connection()
    cur = conn.cursor()

    # get all tables
    cur.execute("""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = 'public'
        ORDER BY table_name;
    """)
    tables = [r[0] for r in cur.fetchall()]

    schema_dict = {}

    for table in tables:
        cur.execute(f"""
            SELECT column_name, data_type
            FROM information_schema.columns
            WHERE table_name = %s;
        """, (table,))
        columns = cur.fetchall()

        cur.execute(f"""
            SELECT
                kcu.column_name,
                ccu.table_name AS foreign_table,
                ccu.column_name AS foreign_column
            FROM
                information_schema.table_constraints AS tc
                JOIN information_schema.key_column_usage AS kcu
                  ON tc.constraint_name = kcu.constraint_name
                JOIN information_schema.constraint_column_usage AS ccu
                  ON ccu.constraint_name = tc.constraint_name
            WHERE
                constraint_type = 'FOREIGN KEY' AND tc.table_name = %s;
        """, (table,))
        fkeys = cur.fetchall()

        schema_dict[table] = {
            "columns": [{"name": c[0], "type": c[1]} for c in columns],
            "foreign_keys": [
                {
                    "column": fk[0],
                    "references": {
                        "table": fk[1],
                        "column": fk[2]
                    }
                } for fk in fkeys
            ]
        }

    cur.close()
    conn.close()

    return schema_dict

In [19]:
extract_schema()

{'customers': {'columns': [{'name': 'customer_id', 'type': 'integer'},
   {'name': 'first_name', 'type': 'character varying'},
   {'name': 'last_name', 'type': 'character varying'},
   {'name': 'email', 'type': 'character varying'},
   {'name': 'phone', 'type': 'character varying'},
   {'name': 'address', 'type': 'character varying'},
   {'name': 'city', 'type': 'character varying'},
   {'name': 'country', 'type': 'character varying'},
   {'name': 'signup_date', 'type': 'date'},
   {'name': 'loyalty_points', 'type': 'integer'}],
  'foreign_keys': []},
 'employees': {'columns': [{'name': 'employee_id', 'type': 'integer'},
   {'name': 'first_name', 'type': 'character varying'},
   {'name': 'last_name', 'type': 'character varying'},
   {'name': 'email', 'type': 'character varying'},
   {'name': 'phone', 'type': 'character varying'},
   {'name': 'department', 'type': 'character varying'},
   {'name': 'position', 'type': 'character varying'},
   {'name': 'hire_date', 'type': 'date'},
   {'n

In [4]:
def schema_to_text(schema_dict):
    text_blocks = []

    for table, info in schema_dict.items():
        columns = ", ".join([f"{col['name']} ({col['type']})" for col in info["columns"]])
        fkeys = [
            f"{fk['column']} → {fk['references']['table']}.{fk['references']['column']}"
            for fk in info["foreign_keys"]
        ]
        fk_text = "; ".join(fkeys) if fkeys else "None"
        table_text = f"Table: {table}\nColumns: {columns}\nForeign Keys: {fk_text}\n"
        text_blocks.append(table_text)

    # Combined text (for embedding)
    full_text = "\n\n".join(text_blocks)

    # Or list form (if your vectorstore stores per-table chunks)
    docs = text_blocks

    return full_text, docs


In [20]:
def schema_to_vector_docs(schema: dict):
    """
    Converts a relational schema (dict) into a list of text documents
    for embedding in a vector store.
    """
    docs = []

    for table_name, table_info in schema.items():
        # --- Table-level document ---
        col_names = [col['name'] for col in table_info['columns']]
        table_doc = {
            "id": f"table::{table_name}",
            "type": "table",
            "text": (
                f"Table '{table_name}' contains data related to {table_name.replace('_', ' ')}. "
                f"It has the following columns: {', '.join(col_names)}."
            ),
            "metadata": {
                "table": table_name,
                "columns": col_names
            }
        }
        docs.append(table_doc)

        # --- Column-level documents ---
        for col in table_info['columns']:
            col_doc = {
                "id": f"column::{table_name}.{col['name']}",
                "type": "column",
                "text": (
                    f"Column '{col['name']}' in table '{table_name}' stores {col['name'].replace('_', ' ')} data. "
                    f"Data type: {col['type']}."
                ),
                "metadata": {
                    "table": table_name,
                    "column": col['name'],
                    "type": col['type']
                }
            }
            docs.append(col_doc)

        # --- Relationship-level documents ---
        for fk in table_info.get('foreign_keys', []):
            src_col = fk['column']
            tgt_table = fk['references']['table']
            tgt_col = fk['references']['column']

            rel_doc = {
                "id": f"relation::{table_name}.{src_col}->{tgt_table}.{tgt_col}",
                "type": "relationship",
                "text": (
                    f"In table '{table_name}', column '{src_col}' is a foreign key referencing "
                    f"column '{tgt_col}' in table '{tgt_table}'. "
                    f"This defines a relationship between '{table_name}' and '{tgt_table}'."
                ),
                "metadata": {
                    "from_table": table_name,
                    "from_column": src_col,
                    "to_table": tgt_table,
                    "to_column": tgt_col
                }
            }
            docs.append(rel_doc)

    return docs


In [22]:
vector_ready_schema = schema_to_vector_docs(extract_schema())

In [31]:
import pprint

pprint.pp(vector_ready_schema)

[{'id': 'table::customers',
  'type': 'table',
  'text': "Table 'customers' contains data related to customers. It has the "
          'following columns: customer_id, first_name, last_name, email, '
          'phone, address, city, country, signup_date, loyalty_points.',
  'metadata': {'table': 'customers',
               'columns': ['customer_id',
                           'first_name',
                           'last_name',
                           'email',
                           'phone',
                           'address',
                           'city',
                           'country',
                           'signup_date',
                           'loyalty_points']}},
 {'id': 'column::customers.customer_id',
  'type': 'column',
  'text': "Column 'customer_id' in table 'customers' stores customer id data. "
          'Data type: integer.',
  'metadata': {'table': 'customers',
               'column': 'customer_id',
               'type': 'integer'}},
 {'id':